In [1]:
import os
import numpy as np
import xarray as xr
import rioxarray
import zarr
from numcodecs import Blosc
import datetime
import glob
import re
import gc
import rasterio.errors


In [2]:
zarr_path = "../../../eodc/products/eodc/clms_vegetation_total_productivity/CLMS.zarr"

start_year = 2017
end_year = 2025
# now = datetime.datetime.now()
# end_year = now.year

# Days of the month you want
days = [1, 11, 21]

# Generate all dates
dates = []
for year in range(start_year, end_year + 1):
    for month in range(1, 13):
        for day in days:
            try:
                dates.append(np.datetime64(datetime.date(year, month, day)))
            except ValueError:
                # In case of invalid dates like Feb 30
                pass

dates = np.array(dates)
dates = dates.astype("datetime64[D]") 
time_int = dates.astype("int64")

In [3]:
# Convert datetime64[D] to int YYYYMMDD
dates_int = (dates.astype('datetime64[D]')
                 .astype('O'))  # convert to Python datetime.date
dates_int = np.array([d.year * 10000 + d.month * 100 + d.day for d in dates_int], dtype='int32')
dates_int

array([20170101, 20170111, 20170121, 20170201, 20170211, 20170221,
       20170301, 20170311, 20170321, 20170401, 20170411, 20170421,
       20170501, 20170511, 20170521, 20170601, 20170611, 20170621,
       20170701, 20170711, 20170721, 20170801, 20170811, 20170821,
       20170901, 20170911, 20170921, 20171001, 20171011, 20171021,
       20171101, 20171111, 20171121, 20171201, 20171211, 20171221,
       20180101, 20180111, 20180121, 20180201, 20180211, 20180221,
       20180301, 20180311, 20180321, 20180401, 20180411, 20180421,
       20180501, 20180511, 20180521, 20180601, 20180611, 20180621,
       20180701, 20180711, 20180721, 20180801, 20180811, 20180821,
       20180901, 20180911, 20180921, 20181001, 20181011, 20181021,
       20181101, 20181111, 20181121, 20181201, 20181211, 20181221,
       20190101, 20190111, 20190121, 20190201, 20190211, 20190221,
       20190301, 20190311, 20190321, 20190401, 20190411, 20190421,
       20190501, 20190511, 20190521, 20190601, 20190611, 20190

In [4]:
store = zarr.storage.LocalStore(zarr_path)
compressor = zarr.codecs.BloscCodec()

root = zarr.group(store=store, zarr_format=3)
dataset = "ST"
ds = root.require_group(dataset)

res = 10

x0 = 4200005
y0 = 2500005

x_extent = np.arange(x0, 4900005, res)
y_extent = np.arange(y0, 3000005, res)


In [5]:
print(ds.path)

ST


In [6]:
time_array = ds.require_array(
    name="time",
    shape=(len(dates_int),),
    dtype="int32",
    chunks=(len(dates_int),),
    dimension_names=("time",),
    attributes={
        "units": "YYYYMMDD",
        "calendar": "noleap",
    },
)
time_array[:] = dates_int

x_array = ds.require_array(
    name="x",
    shape=x_extent.shape,
    dtype="int32",
    chunks=(len(x_extent),),
    dimension_names=("x",),
)
x_array[:] = x_extent

y_array = ds.require_array(
    name="y",
    shape=y_extent.shape,
    dtype="int32",
    chunks=(len(y_extent),),
    dimension_names=("y",),
)
y_array[:] = y_extent


# zarr.consolidate_metadata(store)

In [6]:
time_array

<Array file://../../../eodc/products/eodc/clms_vegetation_total_productivity/CLMS.zarr/ST/time shape=(324,) dtype=int32>

In [7]:
productType = "PPI"

In [8]:
test = rioxarray.open_rasterio("../../../eodc/private/tempearth/CLMS/ST_QFLAG_NEW/ST_20221221T000000_S2_E47N29-03035-010m_V105_QFLAG.tif")

print(test)

<xarray.DataArray (band: 1, y: 10000, x: 10000)> Size: 200MB
[100000000 values with dtype=int16]
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 80kB 4.7e+06 4.7e+06 4.7e+06 ... 4.8e+06 4.8e+06
  * y            (y) float64 80kB 3e+06 3e+06 3e+06 ... 2.9e+06 2.9e+06 2.9e+06
    spatial_ref  int64 8B 0
Attributes:
    TIFFTAG_DATETIME:   2022:12:21 00:00:00
    TIFFTAG_COPYRIGHT:  Copernicus service information 2023
    file_creation:      2023:05:10 09:22:12
    Flag_meaning:       (no data (time series was not processed), Filled (ext...
    Flag_value:         (0, 1, 2, 3, 4, 5)
    input_time_window:  2018-10-01 to 2022-02-29
    TIIFTAG_SOFTWARE:   Timesat : TIMESAT4.1.8
    AREA_OR_POINT:      Area
    _FillValue:         0
    scale_factor:       1.0
    add_offset:         0.0
    long_name:          Seasonal Trajectory Quality Flag


In [8]:
fill_value_PPI = -32768
scale_factor_PPI =  0.0001

attributes_PPI={
        "TIFFTAG_COPYRIGHT": "Copernicus service information 2023",
        "scale_factor": scale_factor_PPI,
        "_FillValue": fill_value_PPI,
        "TIIFTAG_SOFTWARE": "Timesat : TIMESAT4.1.8",
        "PhysRange": "0 to 3",
        "add_offset": 0.0,
        "long_name": "Plant Phenology Index obtained via timesat fitting",
    }


fill_value_QFLAG = 0
scale_factor_QFLAG =  1.0

attributes_QFLAG={
        "TIFFTAG_COPYRIGHT": "Copernicus service information 2023",
        "scale_factor": scale_factor_QFLAG,
        "_FillValue": fill_value_QFLAG,
        "TIIFTAG_SOFTWARE": "Timesat : TIMESAT4.1.8",
        "Flag_value": (0, 1, 2, 3, 4, 5),
        "add_offset": 0.0,
        "long_name": "Seasonal Trajectory Quality Flag",
    }


In [9]:
shape = (len(dates_int), len(y_extent), len(x_extent))
chunk_shape = (1, 10000, 10000)

fv =  globals()[f"fill_value_{productType}"]

if -32768 <= fv < 0:
    dtype = "int16"
elif 0 <= fv <= 65535:
    dtype = "uint16"

In [15]:
data_array = ds.require_array(
    name=productType,
    shape=shape,
    chunks=chunk_shape,
    dtype=dtype,
    fill_value = globals()[f"fill_value_{productType}"],
    compressor=compressor,
    dimension_names=["time", "y", "x"],
    attributes=globals()[f"attributes_{productType}"],
    overwrite=False
)

/home/koenifra/envs/zarr3/lib/python3.11/site-packages/zarr/core/group.py:1119: UserWarning: The `compressor` argument is deprecated. Use `compressors` instead.
  compressors = _parse_deprecated_compressor(


ContainsArrayError: An array exists in store LocalStore('file://../../../eodc/products/eodc/clms_vegetation_total_productivity/CLMS.zarr') at path 'ST/PPI'.

In [10]:
root = zarr.open_group(
    store=store,
    mode="a",
    zarr_format=3,
    use_consolidated=False,   # critical
)

data_array = root["ST"]["PPI"] 

In [11]:
date_to_index = {int(d): i for i, d in enumerate(dates_int)}

x0 = x_extent[0]  # 4_200_000
y0 = y_extent[0]  # 2_500_000
res = 10          # 10 m pro Pixel

path = f"../../../eodc/private/tempearth/CLMS/ST_{productType}/*.tif"


# for y in dates_int[180:191]:
for y in dates_int[213:]:
    print(f"date = {y}")
    print(f"index = {date_to_index[y]}")

    tif_files = sorted([
        f for f in glob.glob(path)
        if "100m" not in f and str(y) in os.path.basename(f)
    ])
        
    for filepath in tif_files:

        print(f"file: {filepath}")

        try:
            file = rioxarray.open_rasterio(filepath)
            tp = file.values.astype("float32")
        except Exception as e:
            print(f"Unexpected error with file: {filepath}")
            print(f"Error: {e}")
            continue


        # Extract both xmin and ymin
        match = re.search(r'E(\d+)N(\d+)', filepath)
        if match:
            # 1) Kachelecken (untere linke Ecke) in Meter
            xmin_corner = int(match.group(1)) * 100000  # z.B. 42 → 4_200_000
            ymin_corner = int(match.group(2)) * 100000

            # 2) Erste Zellmitte in dieser Kachel (wie im TIF)
            xmin_center = xmin_corner + res / 2       # 4_200_000 + 5 = 4_200_005
            ymin_center = ymin_corner + res / 2
        else:
            print("No match found.")
            continue

        ny_tile = tp.shape[1]   # Pixel in y
        nx_tile = tp.shape[2]   # Pixel in x

        xmax_center = xmin_center + (nx_tile - 1) * res
        ymax_center = ymin_center + (ny_tile - 1) * res

        # print(f"tile centers x: {xmin_center} .. {xmax_center}")
        # print(f"tile centers y: {ymin_center} .. {ymax_center}")

        ix_min = int((xmin_center - x0) // res)
        ix_max = ix_min + nx_tile        # slice-Ende ist exklusiv
        iy_min = int((ymin_center - y0) // res)
        iy_max = iy_min + ny_tile

        start_idx = date_to_index[y]
        end_idx   = start_idx + tp.shape[0]

        # print(start_idx)
        
        tp = tp[:, ::-1, :]
        data_array[start_idx:end_idx, iy_min:iy_max, ix_min:ix_max] = tp

        # Free memory
        del tp
        gc.collect()


date = 20221201
index = 213
file: ../../../eodc/private/tempearth/CLMS/ST_PPI/ST_20221201T000000_S2_E42N25-03035-010m_V105_PPI.tif
file: ../../../eodc/private/tempearth/CLMS/ST_PPI/ST_20221201T000000_S2_E42N26-03035-010m_V105_PPI.tif
file: ../../../eodc/private/tempearth/CLMS/ST_PPI/ST_20221201T000000_S2_E42N27-03035-010m_V105_PPI.tif
file: ../../../eodc/private/tempearth/CLMS/ST_PPI/ST_20221201T000000_S2_E42N28-03035-010m_V105_PPI.tif
file: ../../../eodc/private/tempearth/CLMS/ST_PPI/ST_20221201T000000_S2_E43N25-03035-010m_V105_PPI.tif
file: ../../../eodc/private/tempearth/CLMS/ST_PPI/ST_20221201T000000_S2_E43N26-03035-010m_V105_PPI.tif
file: ../../../eodc/private/tempearth/CLMS/ST_PPI/ST_20221201T000000_S2_E43N27-03035-010m_V105_PPI.tif
file: ../../../eodc/private/tempearth/CLMS/ST_PPI/ST_20221201T000000_S2_E43N28-03035-010m_V105_PPI.tif
file: ../../../eodc/private/tempearth/CLMS/ST_PPI/ST_20221201T000000_S2_E44N25-03035-010m_V105_PPI.tif
file: ../../../eodc/private/tempearth/CLMS/ST